# Video2X Codespace Notebook

Adapted from the original [Video2X Colab Notebook](https://github.com/k4yt3x/video2x) for GitHub Codespace environment.

Licensed under the ISC License. Copyright 2022-2024 K4YT3X and contributors.

**Features:**
- 🎯 RealESRGAN and libplacebo filters
- 📁 Local file system (no Google Drive dependency)
- ⚡ CPU/GPU processing support
- 🔧 Interactive configuration

**Note:** This runs in a Codespace environment with local storage.

In [ ]:
# Step 0: System Check (adapted from original Runtime Selection)

# Check GPU availability
import subprocess

print("🔍 Checking system resources...")

try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ GPU detected:")
        print(result.stdout)
    else:
        print("⚠️ No GPU detected - will use CPU processing")
except FileNotFoundError:
    print("⚠️ nvidia-smi not found - CPU processing only")

# Check Video2X installation
try:
    result = subprocess.run(['video2x', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print("\n✅ Video2X is available:")
        print(result.stdout)
    else:
        print("\n❌ Video2X not found - run setup first")
except FileNotFoundError:
    print("\n❌ Video2X not installed - run setup script")

In [ ]:
# Step 1: Install Video2X and Dependencies (adapted for Codespace)

import os
import subprocess

def install_video2x():
    """Install Video2X and dependencies in Codespace environment"""
    
    print("🚀 Installing Video2X and dependencies...")
    
    # Update system
    print("📦 Updating system packages...")
    subprocess.run(['sudo', 'apt-get', 'update'], check=True)
    
    # Install system dependencies
    print("🔧 Installing system dependencies...")
    deps = [
        'curl', 'wget', 'software-properties-common',
        'libvulkan1', 'ffmpeg', 'python3-pip'
    ]
    subprocess.run(['sudo', 'apt-get', 'install', '-y'] + deps, check=True)
    
    # Try to install Video2X deb package (original method)
    try:
        print("📥 Downloading Video2X package...")
        subprocess.run([
            'curl', '-LO', 
            'https://github.com/k4yt3x/video2x/releases/download/6.2.0/video2x-linux-ubuntu2204-amd64.deb'
        ], check=True)
        
        print("🔧 Installing Video2X package...")
        subprocess.run([
            'sudo', 'apt-get', 'install', '-y', 
            './video2x-linux-ubuntu2204-amd64.deb'
        ], check=True)
        
        # Clean up
        os.remove('video2x-linux-ubuntu2204-amd64.deb')
        
    except subprocess.CalledProcessError:
        print("⚠️ Package installation failed, trying pip...")
        subprocess.run(['pip3', 'install', '--user', 'video2x'], check=True)
    
    # Set up Vulkan environment
    os.environ['VK_ICD_FILENAMES'] = '/usr/share/vulkan/icd.d/nvidia_icd.json'
    
    print("✅ Installation complete!")

# Run installation
install_video2x()

In [ ]:
# Step 2: File Management (adapted for local filesystem)

import pathlib
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set up local paths (instead of Google Drive)
workspace_path = pathlib.Path('/workspaces/video2x-codespace')
input_dir = workspace_path / 'input'
output_dir = workspace_path / 'output'
temp_dir = workspace_path / 'temp'

# Ensure directories exist
for dir_path in [input_dir, output_dir, temp_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

input_file = None

def on_dropdown_change(change):
    global input_file
    if change["name"] == "value":
        input_file = pathlib.Path(change["new"])
        print(f"Selected input file: {input_file}")

def upload_new_file(button):
    global input_file
    
    # For Codespace, we'll use file upload widget
    from google.colab import files as colab_files
    
    try:
        # Try Colab-style upload (won't work in Codespace)
        uploaded = colab_files.upload()
    except:
        # Fallback for Codespace
        print("📁 Manual upload: Place your video files in the 'input' directory")
        print(f"   Input directory: {input_dir}")
        print("   Use VS Code file explorer to upload files")
        return
    
    # Check if the user canceled the upload
    if not uploaded:
        print("Upload canceled.")
        return

    for file_name in uploaded.keys():
        # Move uploaded file to input directory
        src_path = pathlib.Path("/tmp") / file_name
        dest_path = input_dir / file_name
        
        # Copy file content
        with open(dest_path, 'wb') as f:
            f.write(uploaded[file_name])
        
        input_file = dest_path
        print(f"Uploaded and set input file: {input_file}")

def refresh_file_list():
    """Refresh the file dropdown with current files"""
    input_files = [f for f in input_dir.iterdir() if f.is_file()]
    return input_files

# Create file selection interface
def create_file_interface():
    # Get current files
    input_files = refresh_file_list()
    
    if input_files:
        file_dropdown = widgets.Dropdown(
            options=input_files, 
            description="Select file:"
        )
        file_dropdown.observe(on_dropdown_change)
        display(file_dropdown)
    else:
        print("📁 No files found in input directory")
        print(f"   Upload files to: {input_dir}")
    
    # Upload button (may not work in Codespace)
    upload_button = widgets.Button(description="Upload New File")
    upload_button.on_click(upload_new_file)
    display(upload_button)
    
    # Manual upload instructions
    print("\n📋 Manual Upload Instructions:")
    print("1. Use VS Code file explorer")
    print(f"2. Navigate to: {input_dir}")
    print("3. Drag and drop video files")
    print("4. Refresh this cell to see new files")

# Display file interface
create_file_interface()

In [ ]:
# Step 3A: Upscale (adapted for Codespace)

import pathlib
import subprocess

# Validation
if 'input_file' not in globals() or input_file is None:
    print("❌ No input file selected. Please run the previous step first.")
    print("💡 Select a file from the dropdown or upload one manually")
else:
    print(f"✅ Input file: {input_file}")

# Configuration parameters (same as original)

# Filter Select
filter_type = "realesrgan"  # Options: "realesrgan", "libplacebo"

# RealESRGAN Options
model = "realesr-animevideov3"  # Options: "realesrgan-plus-anime", "realesrgan-plus", "realesr-animevideov3"
scale = 2  # Scale factor: 2, 3, or 4

# libplacebo Options
shader = "anime4k-v4-a"  # Various anime4k shaders available
width = 3840   # Target width
height = 2160  # Target height

# Advanced Options
codec = "libx264"
preset = "slow"
pixfmt = "auto"
bitrate = 0
crf = 20
loglevel = "info"

# Interactive configuration widgets
def create_config_widgets():
    global filter_type, model, scale, codec, crf
    
    # Filter selection
    filter_widget = widgets.Dropdown(
        options=[('RealESRGAN', 'realesrgan'), ('libplacebo', 'libplacebo')],
        value=filter_type,
        description='Filter:'
    )
    
    # Model selection
    model_widget = widgets.Dropdown(
        options=[
            ('Anime Video v3 (2x-4x)', 'realesr-animevideov3'),
            ('Anime Plus (4x)', 'realesrgan-plus-anime'),
            ('Real-life Plus (4x)', 'realesrgan-plus')
        ],
        value=model,
        description='Model:'
    )
    
    # Scale selection
    scale_widget = widgets.IntSlider(
        value=scale,
        min=2,
        max=4,
        step=1,
        description='Scale:'
    )
    
    # Quality selection
    crf_widget = widgets.IntSlider(
        value=crf,
        min=0,
        max=51,
        step=1,
        description='Quality (CRF):'
    )
    
    def update_filter(change):
        global filter_type
        filter_type = change['new']
    
    def update_model(change):
        global model
        model = change['new']
    
    def update_scale(change):
        global scale
        scale = change['new']
    
    def update_crf(change):
        global crf
        crf = change['new']
    
    filter_widget.observe(update_filter, names='value')
    model_widget.observe(update_model, names='value')
    scale_widget.observe(update_scale, names='value')
    crf_widget.observe(update_crf, names='value')
    
    display(filter_widget)
    display(model_widget)
    display(scale_widget)
    display(crf_widget)
    
    return filter_widget, model_widget, scale_widget, crf_widget

print("⚙️ Configuration:")
widgets = create_config_widgets()

In [ ]:
# Processing execution (run after configuring above)

if input_file is None:
    print("❌ No input file selected. Please select a file first.")
else:
    # Prepare output file path
    output_file = output_dir / f"{input_file.stem}.processed{input_file.suffix}"
    
    if output_file.exists():
        print(f"⚠️ Output file already exists: {output_file}")
        print("Delete it or choose a different name to proceed.")
    else:
        print(f"📤 Output will be written to: {output_file}")
        
        # Build command (same logic as original)
        command = [
            "video2x",
            "--input", str(input_file),
            "--output", str(output_file),
            "--processor", filter_type,
            "--codec", codec,
            "--bit-rate", str(bitrate),
            "--log-level", loglevel,
            "-e", f"preset={preset}",
            "-e", f"crf={crf}"
        ]
        
        # Add filter-specific options
        if filter_type == "realesrgan":
            command.extend([
                "--realesrgan-model", model,
                "--scaling-factor", str(scale)
            ])
        elif filter_type == "libplacebo":
            command.extend([
                "--width", str(width),
                "--height", str(height),
                "--libplacebo-shader", shader
            ])
        
        if pixfmt != "auto":
            command.extend(["--pix-fmt", pixfmt])
        
        print("🚀 Starting Video2X processing...")
        print(f"📋 Command: {' '.join(command)}")
        print("" + "=" * 50)
        
        # Execute command
        try:
            result = subprocess.run(command, check=True, capture_output=False)
            print("\n" + "=" * 50)
            print("✅ Processing completed successfully!")
            print(f"📁 Output saved to: {output_file}")
            
            # Show file size
            if output_file.exists():
                size_mb = output_file.stat().st_size / (1024 * 1024)
                print(f"💾 Output file size: {size_mb:.1f} MB")
                
        except subprocess.CalledProcessError as e:
            print(f"\n❌ Processing failed with error code: {e.returncode}")
            print("Check the logs above for details.")

In [ ]:
# Results Management

def show_results():
    """Display processed files"""
    output_files = [f for f in output_dir.iterdir() if f.is_file()]
    
    if not output_files:
        print("📭 No processed files found.")
        return
    
    print(f"📁 Found {len(output_files)} processed files:")
    print()
    
    for i, file_path in enumerate(output_files, 1):
        size_mb = file_path.stat().st_size / (1024 * 1024)
        print(f"{i:2d}. {file_path.name}")
        print(f"    💾 Size: {size_mb:.1f} MB")
        print(f"    📍 Path: {file_path}")
        print()
    
    print("💡 Download Instructions:")
    print("1. Use VS Code file explorer")
    print(f"2. Navigate to: {output_dir}")
    print("3. Right-click file → Download")

show_results()